In [1]:
from elasticsearch import Elasticsearch
import numpy as np
es = Elasticsearch("http://localhost:9200")

In [7]:
import numpy as np
loaded = np.load('all_fearture.npz')
identity = loaded["identity"]
embedding = loaded["embedding"]

In [8]:
identity.shape

(13236,)

In [9]:
embedding.shape

(13236, 1, 512)

In [5]:
for i in range(len(identity)):
    s = identity[i]
    n = s.split("/")[1]
    token = n.split("_")[0]
    new = token + "_" + s
    identity[i] = new


In [7]:
np.savez_compressed("all_fearture", identity=np.array(identity), embedding=np.array(embedding))

In [4]:
embedding.shape

(6776832,)

In [9]:
es.get(index="face_recognition", id="1")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [face_recognition]', face_recognition, index_or_alias)

In [10]:
index = 0
for i in range(len(identity)):
    doc = {"title_vector": embedding[i][0], "title_name": identity[i]}
    es.index(index="face_recognition", id=index, body=doc)
    index = index + 1

In [6]:
index

13231

In [5]:
len(identity)

13231

In [11]:
target_embedding = embedding[10651][0]

In [12]:
target_embedding.shape

(512,)

In [13]:
query = {
    "size": 1,
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
        "script": {
            "source": "cosineSimilarity(params.queryVector, 'title_vector') + 1.0",
            # "source": "1 / (1 + l2norm(params.queryVector, 'title_vector'))", #euclidean distance
            "params": {
                "queryVector": list(target_embedding)
            }
        }
    }
}}

In [14]:
%time
res = es.search(index="face_recognition", body=query)

CPU times: user 86 µs, sys: 6 µs, total: 92 µs
Wall time: 1.2 ms


In [19]:
res

ObjectApiResponse({'took': 252, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 0.9999994, 'hits': [{'_index': 'face_recognition', '_id': '10651', '_score': 0.9999994, '_source': {'title_vector': [0.723665714263916, -0.5501291155815125, -0.7968102097511292, -1.8583110570907593, 1.5560911893844604, -1.282939076423645, 0.2180427610874176, -0.6070091128349304, -1.266722559928894, 0.8886131048202515, -1.4521992206573486, -0.35941407084465027, -1.2888227701187134, -1.7945934534072876, 0.21987073123455048, 0.5353357791900635, -1.0839601755142212, 1.0335114002227783, 0.12505221366882324, 0.46673187613487244, 0.04920284450054169, 1.7267733812332153, -1.3522263765335083, -0.48848757147789, -0.015657251700758934, -0.5627336502075195, -0.29322922229766846, -1.9309139251708984, -0.4813368320465088, -0.8127191066741943, -0.14615201950073242, -0.0946149230003357, 2.5688095092773438, 1.3984580

In [16]:
identity[10651]

'Lucy_Liu/Lucy_Liu_0003.jpg'

In [15]:
for i in res["hits"]["hits"]:
    candidate_name = i["_source"]["title_name"]
    candidate_score = i["_score"]
    print(candidate_name, ": ", candidate_score)

Lucy_Liu/Lucy_Liu_0003.jpg :  1.9999998


In [21]:
identity[10651]

'Joe_Friedberg/Joe_Friedberg_0001.jpg'

In [31]:
index = np.where(identity == 'Cruise/Tom_Cruise_0002.jpg')

In [32]:
index

(array([10651]),)

In [17]:
import cv2

In [18]:
img = cv2.imread("celeb_data/Aaron_Eckhart/Aaron_Eckhart_0001.jpg")

In [23]:
np.unique(img)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [22]:
while 1:    
    cv2.imshow("name",img)
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break